# FEgrow - congeneric ligands from a template

### Developers: Mat Bieniek, Ben Cree, Rachael Pirie, Josh Horton

* Add R-groups in user-defined positions
* Output ADMET properties
* Perform constrained optimisation
* Score poses
* Send output to free energy calculations

Open source, customisable, fast/easy to use.

In [ ]:
import copy

import prody
from rdkit import Chem

import fegrow
from fegrow import RGroups

# Prepare the ligand template

In [ ]:
# protonate the template structure with open babel
!obabel sarscov2/lig.pdb -O sarscov2/coreh.sdf -p 7

In [ ]:
# load the protonated ligand
init_mol = Chem.SDMolSupplier('sarscov2/coreh.sdf', removeHs=False)[0]

# get the FEgrow representation of the rdkit Mol
template_mol = fegrow.RMol(init_mol)

In [ ]:
# select the atom to be replaced with the R group
template_mol.rep2D(idx=True, size=(500, 500))

In [ ]:
# using the 2D drawing, select a hydrogen index for the R-group
attachment_index = [40]

In [ ]:
# interactive selection
RGroups

In [ ]:
# retrieve the interactively selected groups
interactive_rgroups = RGroups.get_selected()

# you can also directly access the built-in dataframe programmatically
groups = RGroups.dataframe
R_group_ethanol = groups.loc[groups['Name']=='ethanol']['Mol'].values[0]
R_group_cyclopropane = groups.loc[groups['Name'] == 'cyclopropane' ]['Mol'].values[0]

# add your own R-groups files
R_group_methanol = Chem.MolFromMolFile('manual_rgroups/methanol-r.mol', removeHs=False)

# make a list of R-group molecule
selected_rgroups = [R_group_methanol, R_group_ethanol, R_group_cyclopropane] + interactive_rgroups

# Build a congeneric series

In [ ]:
# build the new molecules by merging the template with the R-groups
rmols = fegrow.build_molecules(template_mol, 
                               attachment_index, 
                               selected_rgroups)

In [ ]:
rmols.rep2D()

In [ ]:
rmols[0].rep3D()

In [ ]:
rmols.toxicity()

In [ ]:
# note that you can add flexibility
rmols.generate_conformers(num_conf=10, 
                         minimum_conf_rms=0.5, 
                        )
                        # flexible=[39, 18, 40, 16, 17, 19, 5, 6, 36, 2, 1, 33, 3, 4, 35, 34, 15, 38])

In [ ]:
#rmol.rep3D(template=True)

### Optimise conformers in context of protein

In [ ]:
# load the complex with the ligand
sys = prody.parsePDB('sarscov2/7l10.pdb')

# remove any unwanted molecules
rec = sys.select('not (nucleic or hetatm or water)')

# save the processed protein
prody.writePDB('rec.pdb', rec)

# fix the receptor file 
fegrow.fix_receptor("rec.pdb", "rec_h.pdb")

# load back into prody
rec_fix = prody.parsePDB("rec_h.pdb")

In [ ]:
rmols[0].rep3D(prody=rec_fix)

In [ ]:
# filter by the protein clashes
rmols.removeConfsClashingWithProdyProt(rec_fix)

In [ ]:
rmols[0].rep3D(prody=rec_fix)

In [ ]:
# opt_mol, energies
energies = rmols.optimise_in_receptor(
    receptor_file="rec_h.pdb", 
    ligand_force_field="openff", 
    use_ani=False,
    sigma_scale_factor=0.8,
    relative_permittivity=4,
    water_model = None
)

In [ ]:
rmols[0].rep3D()

In [ ]:
rmols[0].to_file("optimised_coords.pdb")

In [ ]:
print(energies)

In [ ]:
# select the best 5 performing conformers
final_energies = rmols.sort_conformers(energy_range=5)

In [ ]:
rmols[0].to_file("best_conformers0.pdb")
rmols[1].to_file("best_conformers1.pdb")

In [ ]:
print(final_energies)

In [ ]:
# score the conformers with Gnina
CNNscores = rmols.gnina(receptor_file="rec_h.pdb") 
CNNscores